In [122]:
import numpy as np

def lanczos_algorithm(L, x, m):
    """
    Implements a simple Lanczos algorithm for a symmetric matrix L.
    
    Parameters:
    L (numpy array): Symmetric matrix (n x n)
    x (numpy array): Initial vector (n x 1)
    m (int): Number of Lanczos iterations (dimension of Krylov subspace)
    
    Returns:
    V (numpy array): Orthonormal basis vectors (n x m)
    alpha (numpy array): Diagonal elements (m x 1)
    beta (numpy array): Subdiagonal elements (m-1 x 1)
    """
    n = L.shape[0]  # Dimension of the matrix
    V = np.zeros((n, m))  # To store basis vectors
    alpha = np.zeros(m)  # Diagonal elements
    beta = np.zeros(m-1)  # Subdiagonal elements
    
    # Step 1: Initialize v_1 = x / ||x||
    beta_1 = np.linalg.norm(x)
    if beta_1 == 0:
        raise ValueError("Initial vector x cannot be zero.")
    v_1 = x / beta_1
    V[:, 0] = v_1  # Store the first basis vector
    
    # Step 2: Initialize w_0 = 0 (for j=1, v_0 is considered 0)
    w = np.zeros(n)
    
    # Step 3: Lanczos iteration for j = 1 to m-1
    for j in range(m):
        # Compute w'_j = L v_j
        w_prime = L @ V[:, j]
        
        # Compute alpha_j = (w'_j)^T v_j
        alpha[j] = np.dot(w_prime, V[:, j])
        
        # Compute w_j = w'_j - alpha_j v_j - beta_j v_{j-1}
        w = w_prime - alpha[j] * V[:, j]
        if j > 0:
            w -= beta[j-1] * V[:, j-1]
        
        # Compute beta_{j+1} = ||w_j||
        if j < m-1:  # We only need beta up to m-1
            beta[j] = np.linalg.norm(w)
            if beta[j] == 0:
                print(f"Breakdown at iteration {j+1}: w_{j+1} is zero.")
                break
            # Compute v_{j+1} = w_j / beta_{j+1}
            V[:, j+1] = w / beta[j]
    
    return V, alpha, beta

def construct_tridiagonal_matrix(alpha, beta):
    """
    Heter: Construct the tridiagonal matrix \( T_m \) using the alpha and beta coefficients.
    
    Parameters:
    alpha (numpy array): Diagonal elements (m x 1)
    beta (numpy array): Subdiagonal elements (m-1 x 1)
    
    Returns:
    T (numpy array): Tridiagonal matrix (m x m)
    """
    m = len(alpha)
    T = np.zeros((m, m))
    
    # Fill diagonal with alpha
    for i in range(m):
        T[i, i] = alpha[i]
    
    # Fill subdiagonal and superdiagonal with beta
    for i in range(m-1):
        T[i, i+1] = beta[i]  # Superdiagonal
        T[i+1, i] = beta[i]  # Subdiagonal (symmetric)
    
    return T

def check_lanczos_equation(L, V, alpha, beta, tol=1e-10):
    """
    Verifies if V^T L V = T holds within a tolerance.
    
    Parameters:
    L (numpy array): Original symmetric matrix (n x n)
    V (numpy array): Orthonormal basis vectors (n x m)
    alpha (numpy array): Diagonal elements (m x 1)
    beta (numpy array): Subdiagonal elements (m-1 x 1)
    tol (float): Tolerance for checking equality
    
    Returns:
    bool: True if the equation holds within tolerance, False otherwise
    """
    # Construct the tridiagonal matrix T
    T = construct_tridiagonal_matrix(alpha, beta)
    
    # Compute V^T L V
    VTLV = V.T @ L @ V
    
    # Check if V^T L V is approximately equal to T
    difference = np.abs(VTLV - T)
    max_diff = np.max(difference)
    
    print("Computed V^T L V:")
    print(VTLV)
    print("\nTridiagonal matrix T:")
    print(T)
    print("\nMaximum difference between V^T L V and T:", max_diff)
    
    return max_diff < tol

# Create a small symmetric matrix L (for testing)
# n = 4
# L = np.array([
#     [4, 1, 0, 0],
#     [1, 3, 1, 0],
#     [0, 1, 2, 1],
#     [0, 0, 1, 1]
# ])

# # Larger example usage
# n = 20
# # Create a random symmetric positive definite matrix L
# # np.random.seed(43)  # For reproducibility
# A = np.random.rand(n, n)
# L = (A + A.T) / 2   # Ensure positive definiteness

np.random.seed(43)
n = 20  # Matrix size
epsilon = 0.01  # Perturbation strength

# Diagonal matrix
D = np.eye(n)  # Identity (eigenvalues = 1)

# Symmetric perturbation
E = np.random.randn(n, n) * epsilon
E = (E + E.T) / 2  # Ensure symmetry

# Construct matrix
L = D + E


# Initial vector x
x = np.ones(n)

# Number of iterations
m = 4

# Run Lanczos algorithm
V, alpha, beta = lanczos_algorithm(L, x, m)

T = construct_tridiagonal_matrix(alpha, beta)
# Check if V^T L V = T holds
is_valid = check_lanczos_equation(L, V, alpha, beta)

print("\nDoes V^T L V = T hold within tolerance?", is_valid)

Computed V^T L V:
[[ 9.98029426e-01  2.73997867e-02 -2.38697950e-15 -2.08860707e-14]
 [ 2.73997867e-02  1.00636629e+00  3.06296261e-02 -4.12170298e-15]
 [-2.41473508e-15  3.06296261e-02  1.00105024e+00  1.95566336e-02]
 [-2.08236206e-14 -4.17721413e-15  1.95566336e-02  1.00508636e+00]]

Tridiagonal matrix T:
[[0.99802943 0.02739979 0.         0.        ]
 [0.02739979 1.00636629 0.03062963 0.        ]
 [0.         0.03062963 1.00105024 0.01955663]
 [0.         0.         0.01955663 1.00508636]]

Maximum difference between V^T L V and T: 2.114627917215728e-14

Does V^T L V = T hold within tolerance? True


In [123]:
eigenvalues_L, _ = np.linalg.eig(L)
eigenvalues_T, _ = np.linalg.eig(T)

In [124]:
eigenvalues_L

array([0.94618255, 0.95666692, 0.96099278, 1.05806662, 0.97065047,
       0.97724224, 1.04999927, 0.98307938, 1.04595615, 1.03914346,
       1.04020204, 1.02952795, 0.99107215, 1.02434796, 1.02002779,
       1.013177  , 0.99689368, 1.00779733, 1.00420663, 0.99988022])

In [125]:
eigenvalues_T

array([0.95913526, 1.04708066, 0.98954226, 1.01477415])

In [14]:
import numpy as np

def unrolled_lanczos_algorithm(L, x, m, gamma=None):
    """
    Implements an unrolled Lanczos algorithm with learnable gamma parameters.
    The beta_j v_{j-1} term is subtracted after scaling by gamma_j.
    
    Parameters:
    L (numpy array): Symmetric matrix (n x n)
    x (numpy array): Initial vector (n x 1)
    m (int): Number of Lanczos iterations (dimension of Krylov subspace)
    gamma (numpy array): Learnable parameters (m-1 x 1), defaults to ones if None
    
    Returns:
    V (numpy array): Basis vectors (n x m)
    alpha (numpy array): Diagonal elements (m x 1)
    beta (numpy array): Subdiagonal elements (m-1 x 1)
    """
    n = L.shape[0]
    V = np.zeros((n, m))
    alpha = np.zeros(m)
    beta = np.zeros(m-1)
    
    if gamma is None:
        gamma = np.ones(m)
    else:
        assert len(gamma) == m-1, "gamma must have length m-1"

        
    # Initialize v_1 = x / ||x||
    beta_1 = np.linalg.norm(x)
    if beta_1 == 0:
        raise ValueError("Initial vector x cannot be zero.")
    v_1 = x / beta_1
    V[:, 0] = v_1
    
    w = np.zeros(n)
    
    for j in range(m):
        w_prime = L @ V[:, j]
        alpha[j] = np.dot(w_prime, V[:, j])
        w = w_prime - alpha[j] * V[:, j]
        if j < m-1:
            w = gamma[j] * w
            if j > 0:
                w -= beta[j-1] * V[:, j-1]
            beta[j] = np.linalg.norm(w)
            if beta[j] == 0:
                print(f"Breakdown at iteration {j+1}: w_{j+1} is zero.")
                break
            V[:, j+1] = w / beta[j]
    
    return V, alpha, beta, gamma

def construct_tridiagonal_matrix(alpha, beta, gamma):
    """
    Construct the tridiagonal matrix T with asymmetric off-diagonals and scaled diagonal.
    Diagonal: alpha_i / gamma_i
    Upper diagonal: beta_i / gamma_i
    Lower diagonal: beta_i / gamma_{i-1}
    """
    m = len(alpha)
    T = np.zeros((m, m))
    
    # Fill diagonal with alpha_i / gamma_i
    for i in range(m):  # Use gamma[i] for diagonal elements up to m-1
        T[i, i] = alpha[i]
    
    # Fill off-diagonals with beta_i / gamma_i and beta_i / gamma_{i-1}
    for i in range(m-1):
        if i < m-2:
            T[i, i+1] = beta[i] / gamma[i+1]  # Upper diagonal
        
        T[i+1, i] = beta[i] / gamma[i]  # Lower diagonal
    
    return T[:m, :m]

def check_tridiagonalization(L, V, alpha, beta, gamma, tol=1e-9):
    """
    Checks if T = V_left_inv L V holds, where V_left_inv is a left inverse of V.
    
    Parameters:
    L (numpy array): Original symmetric matrix (n x n)
    V (numpy array): Basis vectors (n x m)
    alpha (numpy array): Diagonal elements (m x 1)
    beta (numpy array): Subdiagonal elements (m-1 x 1)
    gamma (numpy array): Learnable parameters (m-1 x 1)
    tol (float): Tolerance for checking equality
    
    Returns:
    bool: True if the equation holds within tolerance, False otherwise
    """
    m, n = len(gamma), V.shape[0]
    
    T = construct_tridiagonal_matrix(alpha, beta, gamma)
    
    # Check rank of V to ensure full column rank
    rank = np.linalg.matrix_rank(V)
    print(f"Rank of V: {rank}, expected: {m}")
    if rank < m:
        print("Warning: V is rank-deficient, left inverse may not exist.")
        return False
    
    # Compute the left inverse V_left_inv = (V^T V)^{-1} V^T
    V_T_V = V.T @ V
    V_left_inv = np.linalg.inv(V_T_V) @ V.T
    
    # Verify V_left_inv V = I_m
    identity_check = V_left_inv @ V
    print("\nV_left_inv @ V (should be identity):")
    print(identity_check)
    max_identity_diff = np.max(np.abs(identity_check - np.eye(m+1)))
    print("Maximum difference from identity:", max_identity_diff)
    
    # Compute V_left_inv L V
    VLV = V_left_inv @ L @ V
    difference = np.abs(VLV[:-1, :-1] - T[:-1, :-1])
    max_diff = np.max(difference)
    
    print("\nComputed V_left_inv L V:")
    print(VLV)
    print("\nTridiagonal matrix T (asymmetric with scaled diagonal):")
    print(T)
    print("\nMaximum difference between V_left_inv L V and T:", max_diff)
    
    return max_diff < tol

# # Example usage
# n = 4
# L = np.array([
#     [4, 1, 0, 0],
#     [1, 3, 1, 0],
#     [0, 1, 2, 1],
#     [0, 0, 1, 1]
# ])
# x = np.ones(n)
# m = 3
# gamma = np.array([1.5, 0.5])  # Non-default gamma values

# Larger example usage
n = 7
# Create a random symmetric positive definite matrix L
np.random.seed(42)  # For reproducibility
A = np.random.rand(n, n)
L = (A + A.T) / 2 + n * np.eye(n)  # Ensure positive definiteness

# Initial random vector x
x = np.random.rand(n)

# Number of iterations
m = 4

# Define gamma values (m-1 = 4 values)
gamma = np.array([1.2, 0.8, 1.5])  # Varied gamma values to test

gamma = np.append(gamma, 1.0)

V, alpha, beta, gamma = unrolled_lanczos_algorithm(L, x, m+1, gamma)
# print(V.shape)
# print(alpha.shape)
# print(beta.shape)
# print(gamma.shape)

T = construct_tridiagonal_matrix(alpha, beta, gamma)

# Check tridiagonalization with left inverse
is_valid = check_tridiagonalization(L, V, alpha, beta, gamma)

print("\nDoes T = V_left_inv L V hold within tolerance?", is_valid)

Rank of V: 5, expected: 4

V_left_inv @ V (should be identity):
[[ 1.00000000e+00  3.45010258e-12 -2.30969128e-11  1.45396758e-11
   8.95109408e-12]
 [-1.14618037e-12  1.00000000e+00 -8.04482939e-12  5.14458971e-12
   3.44410808e-12]
 [-4.10275946e-12  1.34025082e-11  1.00000000e+00  5.63707960e-12
  -1.18362888e-12]
 [-1.74038995e-11  7.62137776e-12 -1.14186304e-11  1.00000000e+00
  -8.01043467e-13]
 [-2.82173563e-12  1.21991463e-12  6.78058916e-13  2.04343524e-12
   1.00000000e+00]]
Maximum difference from identity: 2.4125368369709577e-11

Computed V_left_inv L V:
[[ 9.88880771e+00  1.40729170e+00 -1.66909193e-10  9.85266548e-11
  -1.37277239e+01]
 [ 9.38194464e-01  7.08931486e+00  4.26216162e-01  3.53438144e-11
  -5.28510933e+00]
 [-2.81553100e-11  7.99155304e-01  8.14294546e+00  2.03315060e+00
  -1.20661076e+01]
 [-1.64977049e-10  2.02769812e-11  1.35543373e+00  9.41135230e+00
  -1.29063793e+01]
 [-2.67720530e-11  5.23387240e-12  8.80543523e-12  9.65965877e-01
   3.57212628e+00]]



In [18]:
eigenvalues_L, _ = np.linalg.eig(L)
eigenvalues_T, _ = np.linalg.eig(T)

In [19]:
eigenvalues_L

array([10.18664835,  7.70412511,  6.33408316,  6.30303534,  6.74752475,
        7.17940683,  7.01494392])

In [20]:
eigenvalues_T

array([ 8.01884404,  6.34072336,  7.30430731, 10.26724546, 10.6201442 ])

In [14]:
V

array([[ 0.69565804, -0.33292016, -0.62177506,  0.02707787,  0.58752291],
       [ 0.17690897,  0.72917287, -0.50109876, -0.247693  ,  0.42392031],
       [ 0.45560656,  0.2594094 ,  0.00362792, -0.68907818, -0.06610968],
       [ 0.52487338, -0.07156013, -0.56033477, -0.07816002,  0.67124463],
       [ 0.0411546 ,  0.5339073 ,  0.21978989, -0.67600092, -0.14201605]])

In [17]:
    V_T_V = V.T @ V
    V_left_inv = np.linalg.inv(V_T_V) @ V.T
    
    # Compute V_left_inv L V
    VLV = V_left_inv @ L @ V
    difference = np.abs(VLV - T)
    max_diff = np.max(difference)


NameError: name 'T' is not defined

In [14]:
V_left_inv = np.linalg.pinv(V)
V_left_inv

array([[ 1.40566038e+00,  7.26415094e-01, -1.19811321e+00,
         1.06603774e+00],
       [ 4.08248290e-01,  4.08248290e-01, -3.53096865e-17,
        -8.16496581e-01],
       [ 1.01068224e+00,  2.52670559e-01, -1.89502919e+00,
         6.31676397e-01]])

In [17]:
V_T_V = V.T @ V
V_left_inv = np.linalg.inv(V_T_V) @ V.T

In [18]:
V_left_inv @ V

array([[ 1.00000000e+00,  2.22044605e-16,  7.77156117e-16],
       [ 2.77555756e-17,  1.00000000e+00, -1.38777878e-17],
       [-4.44089210e-16, -5.55111512e-17,  1.00000000e+00]])